Compute
============================

Now that we have datasets added to our Bundle, our next step is to run the forward model and compute a synthetic model for each of these datasets.

Setup
-----------------------------

Let's first make sure we have the latest version of PHOEBE 2.4 installed (uncomment this line if running in an online notebook session such as colab).

In [1]:
#!pip install -I "phoebe>=2.4,<2.5"

Now we'll import our packages and initialize the default PHOEBE bundle.

In [2]:
import phoebe
from phoebe import u # units
import numpy as np
import matplotlib.pyplot as plt

logger = phoebe.logger()

b = phoebe.default_binary()

And we'll attach some dummy datasets.  See the [datasets tutorial](datasets.ipynb) for more details.

In [3]:
b.add_dataset('orb', 
              compute_times=phoebe.linspace(0,10,10), 
              dataset='orb01')

b.add_dataset('lc', 
              compute_times=phoebe.linspace(0,1,101),
              dataset='lc01')

<ParameterSet: 78 parameters | contexts: compute, dataset, figure, constraint>

Default Compute Options
----------------------------------

Any default Bundle already has a set of default compute options to run the backend for PHOEBE 2.  In most cases, you can just edit the options in this default set of compte options.


In [4]:
print(b.computes)

['phoebe01']


In [5]:
print(b.filter(context='compute'))

ParameterSet: 19 parameters
     sample_from@phoebe01@compute: []
        comments@phoebe01@compute: 
   dynamics_method@phoebe01@co...: keplerian
            ltte@phoebe01@compute: False
    irrad_method@phoebe01@compute: horvat
   boosting_method@phoebe01@co...: none
   eclipse_method@phoebe01@com...: native
   horizon_method@phoebe01@com...: boolean
   mesh_method@primary@phoebe0...: marching
   mesh_method@secondary@phoeb...: marching
   ntriangles@primary@phoebe01...: 1500
   ntriangles@secondary@phoebe...: 1500
   distortion_method@primary@p...: roche
   distortion_method@secondary...: roche
     atm@primary@phoebe01@compute: ck2004
   atm@secondary@phoebe01@compute: ck2004
   enabled@orb01@phoebe01@compute: True
    enabled@lc01@phoebe01@compute: True
      fti_method@phoebe01@compute: none


In [6]:
b.set_value(qualifier='irrad_method', value='none')

Adding Compute Options
--------------------------------------

In other cases, we may want to manually add additional sets of compute options.
This syntax should look very familiar by now, it takes a function (or the name of a recognized function in [phoebe.parameters.compute](../api/phoebe.parameters.compute.md)) and then any
kwargs to set in that ParameterSet, passed to [b.add_compute](../api/phoebe.frontend.bundle.Bundle.add_compute.md).

Let's say that we want to create two sets of compute options - in this example, we'll create one called 'preview' which will cut some corners to quickly get us a model, and one called 'detailed' which will get a much more precise model but likely take longer.  As with other tags, the string you provide for the compute tag is up to you (so long as it doesn't raise an error because it conflicts with other tags).

In [7]:
b.add_compute(phoebe.compute.phoebe, compute='preview', irrad_method='none')

<ParameterSet: 35 parameters | datasets: lc01, _default, orb01>

In [8]:
print(b.filter(compute='preview', context='compute'))

ParameterSet: 19 parameters
      sample_from@preview@compute: []
         comments@preview@compute: 
   dynamics_method@preview@com...: keplerian
             ltte@preview@compute: False
     irrad_method@preview@compute: none
   boosting_method@preview@com...: none
   eclipse_method@preview@compute: native
   horizon_method@preview@compute: boolean
    enabled@orb01@preview@compute: True
     enabled@lc01@preview@compute: True
   mesh_method@primary@preview...: marching
   mesh_method@secondary@previ...: marching
   ntriangles@primary@preview@...: 1500
   ntriangles@secondary@previe...: 1500
   distortion_method@primary@p...: roche
   distortion_method@secondary...: roche
      atm@primary@preview@compute: ck2004
    atm@secondary@preview@compute: ck2004
       fti_method@preview@compute: none


In [9]:
b.add_compute('phoebe', compute='detailed', irrad_method='wilson')

<ParameterSet: 35 parameters | datasets: lc01, _default, orb01>

In [10]:
print(b.get_compute('detailed'))

ParameterSet: 19 parameters
     sample_from@detailed@compute: []
        comments@detailed@compute: 
   dynamics_method@detailed@co...: keplerian
            ltte@detailed@compute: False
    irrad_method@detailed@compute: wilson
   boosting_method@detailed@co...: none
   eclipse_method@detailed@com...: native
   horizon_method@detailed@com...: boolean
   enabled@orb01@detailed@compute: True
    enabled@lc01@detailed@compute: True
   mesh_method@primary@detaile...: marching
   mesh_method@secondary@detai...: marching
   ntriangles@primary@detailed...: 1500
   ntriangles@secondary@detail...: 1500
   distortion_method@primary@d...: roche
   distortion_method@secondary...: roche
     atm@primary@detailed@compute: ck2004
   atm@secondary@detailed@compute: ck2004
      fti_method@detailed@compute: none


Editing Compute Options
-------------------------------------

### Backend-Specific Compute Options

Most of the parameters in the compute options are specific to the backend being used.  Here, of course, we're using the PHOEBE 2.0 backend - but for details on other backends see the [Advanced: Alternate Backends Tutorial](./alternate_backends.ipynb).

The PHOEBE compute options are described in the tutorial on their relevant dataset types:

* [Light Curves/Fluxes (lc)](./LC.ipynb)
* [Radial Velocities (rv)](./RV.ipynb)
* [Line Profiles (lp)](./LP.ipynb)
* [Orbits (orb)](./ORB.ipynb)
* [Meshes (mesh)](./MESH.ipynb)

### Enabling/Disabling Datasets

By default, synthetic models will be created for all datasets in the Bundle when [run_compute](../api/phoebe.frontend.bundle.Bundle.run_commpute.md) is called.  But you can disable a dataset to have run_compute ignore that dataset.  This is handled by a [BoolParameter](../api/phoebe.parameters.BoolParameter.md) with the qualifier 'enabled' - and has a copy that lives in each set of compute options

Let's say we wanted to compute the orbit but not light curve - so we want to set enabled@lc01:

In [11]:
print(b.filter(qualifier='enabled', dataset='lc01'))

ParameterSet: 3 parameters
    enabled@lc01@phoebe01@compute: True
     enabled@lc01@preview@compute: True
    enabled@lc01@detailed@compute: True


as you can see, there is a copy for both of our compute options ('preview' and 'detailed').

If we know which set of compute options we'll be using, or only want to enable/disable for a given set, then we can do that (we could also use [b.disable_dataset](../api/phoebe.frontend.bundle.Bundle.disable_dataset.md) and [b.enable_dataset](../api/phoebe.frontend.bundle.Bundle.enable_dataset.md):

In [12]:
b.set_value(qualifier='enabled', dataset='lc01', compute='preview', value=False)
print(b.filter(qualifier='enabled', dataset='lc01'))

ParameterSet: 3 parameters
    enabled@lc01@phoebe01@compute: True
     enabled@lc01@preview@compute: False
    enabled@lc01@detailed@compute: True


or to enable/disable a dataset for all sets of compute options, we can use the [set_value_all](../api/phoebe.parameters.ParameterSet.set_value_all.md) method:

In [13]:
b.set_value_all('enabled@lc01', True)
print(b.filter(qualifier='enabled', dataset='lc01'))

ParameterSet: 3 parameters
    enabled@lc01@phoebe01@compute: True
     enabled@lc01@preview@compute: True
    enabled@lc01@detailed@compute: True


If the enabled parameter is missing for a set of compute options - it is likely that that particular backend does not support that dataset type.

Running Compute
-----------------------

[run_compute](../api/phoebe.frontend.bundle.Bundle.run_coompute.md) takes arguments for the compute tag as well as the model tag for the resulting synthetic model(s).  

You do not need to provide the compute tag if only 0 or 1 set of compute options exist in the Bundle.  If there are no compute options, the default PHOEBE 2 options will be added on your behalf and used.  If there is a single set of compute options, those will be assumed.  In our case, we have two compute options in the Bundle (with tags 'preview' and 'detailed') so we *must* provide an argument for compute.

If you do not provide a tag for the model, one will be created for you called 'latest'.  Note that the 'latest' model will be overwritten without throwing any errors, whereas other named models can only be overwritten if you pass `overwrite=True` (see the [run_compute API docs](../api/phoebe.frontend.bundle.Bundle.run_compute.md) for details).  In general, though, if you want to maintain the results from previous calls to run_compute, you must provide a NEW model tag.

In [14]:
b.run_compute(compute='preview')

100%|██████████| 110/110 [00:01<00:00, 83.92it/s]


<ParameterSet: 17 parameters | kinds: lc, orb>

In [15]:
print(b.models)

['latest']


### Storing/Tagging Models

Now let's compute models for three different 'versions' of parameters.  By providing a model tag, we can keep the synthetics for each of these different runs in the bundle - which will be handy later on for plotting and comparing models.

In [16]:
b.set_value(qualifier='incl', kind='orbit', value=90)
b.run_compute(compute='preview', model='run_with_incl_90') 

100%|██████████| 110/110 [00:01<00:00, 80.79it/s]


<ParameterSet: 17 parameters | kinds: lc, orb>

In [17]:
b.set_value(qualifier='incl', kind='orbit', value=85)
b.run_compute(compute='preview', model='run_with_incl_85')

100%|██████████| 110/110 [00:01<00:00, 79.64it/s]


<ParameterSet: 17 parameters | kinds: lc, orb>

In [18]:
b.set_value(qualifier='incl', kind='orbit', value=80)
b.run_compute(compute='preview', model='run_with_incl_80')

100%|██████████| 110/110 [00:06<00:00, 16.78it/s]


<ParameterSet: 17 parameters | kinds: lc, orb>

We will now have three new sets of synthetics which can be compared, plotted, or removed.

In [19]:
print(b.models)

['latest', 'run_with_incl_90', 'run_with_incl_85', 'run_with_incl_80']


To remove a model, call [remove_model](../api/phoebe.frontend.bundle.Bundle.remove_model).

In [20]:
b.remove_model('latest')

<ParameterSet: 17 parameters | kinds: lc, orb>

In [21]:
print(b.models)

['latest', 'run_with_incl_90', 'run_with_incl_85', 'run_with_incl_80']


Accessing Synthetics from Models
----------------------------------------

The synthetics can be accessed by their dataset and model tags.

In [22]:
b.filter(model='run_with_incl_90')

<ParameterSet: 19 parameters | contexts: model, figure>

In [23]:
b.filter(component='primary', model='run_with_incl_90')

<ParameterSet: 7 parameters | qualifiers: vws, us, vus, times, vs, ws, vvs>

In [24]:
b.get_parameter(qualifier='us', component='primary', model='run_with_incl_90')

<Parameter: us=[ 1.62265701e-16 -1.70338717e+00 -2.60974055e+00 ...
  2.60974055e+00  1.70338717e+00  2.07569028e-14] solRad | keys: description, value, default_unit, visible_if, required_shape, copy_for, readonly, advanced, latexfmt>

In [25]:
b.get_value(qualifier='us', dataset='orb01', component='primary', model='run_with_incl_90')[:10]

array([ 1.62265701e-16, -1.70338717e+00, -2.60974055e+00, -2.29496732e+00,
       -9.06353380e-01,  9.06353380e-01,  2.29496732e+00,  2.60974055e+00,
        1.70338717e+00,  2.07569028e-14])

For more details about the resulting Parameters in the model context, see the tutorial on the relevant dataset types:

* [Light Curves/Fluxes (lc)](./LC.ipynb)
* [Radial Velocities (rv)](./RV.ipynb)
* [Line Profiles (lp)](./LP.ipynb)
* [Orbits (orb)](./ORB.ipynb)
* [Meshes (mesh)](./MESH.ipynb)

Next
----------

Next up: let's start [plotting](./plotting.ipynb) our synthetic model.

Or look at any of these advanced topics related to computing observables:
* [Advanced: Compute Times & Phases](compute_times_phases.ipynb)
* [Advanced: Running Multiple Compute Options Simulataneously](compute_multiple.ipynb)
* [Advanced: Alternate Backends](alternate_backends.ipynb)
* [Advanced: Detaching from Run Compute](detach.ipynb)